In [ ]:
import sklearn
from sklearn.linear_model import Ridge
from scipy.io import loadmat
import functionList
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
raw = loadmat(r"C:\Users\parkf\Desktop\summer2021\eegData\for_Barry\524R.mat") #change this to try different one off files
X = raw['data']  
mask = raw['mask']
l = X.shape[0]

In [ ]:
memVec = [4, 8, 16]
ridgeVec = [0, 1, 10]
ridgeVecNormal = np.dot(ridgeVec,l) #use a regularization parameter that is based on length
cvNumber = 3

s = (len(memVec),len(ridgeVec),cvNumber)
error = np.zeros(s)
for memNumber in memVec:
    featureMat = functionList.buildFeatureMat(X,memNumber)
    featureMat = functionList.removeTrash(featureMat, mask, memNumber)
    xClean = functionList.removeTrash(X, mask, memNumber)
    for regularizer in ridgeVecNormal: 
        RR = Ridge(alpha=regularizer)
        crossVal = sklearn.model_selection.KFold(n_splits=cvNumber)
        cvRun = 0
        for train, test in crossVal.split(featureMat):
            RR.fit(featureMat[train,:],xClean[train])
            pred = RR.predict(featureMat[test])
            error[memVec.index(memNumber), ridgeVec.index(regularizer/l), cvRun] = functionList.findMSE(pred, xClean[test])
            cvRun = cvRun + 1

In [ ]:
errNoCv = np.mean(error, 2)
titleString = f"Memory vs MSE"
plt.title(titleString)
for i in range(len(ridgeVec)):
    plt.plot(memVec[:], errNoCv[:, i], ".", label = f"lambda = {ridgeVec[i]}") ## figure out columns to pass
    plt.xlabel("Number of memory points")
    plt.ylabel("MSE")
plt.legend()
plt.show()